#데이터 전처리

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv("/content/drive/Shareddrives/21-01 데마팀 화이팅/크롤링 결과/pre_dataset.csv",encoding="utf-8-sig")

In [ ]:
#https://towardsdatascience.com/keyword-extraction-with-bert-724efca412ea
#<Keyword Extraction with BERT>

!pip install sentence-transformers

In [ ]:
df.isnull().sum()

Unnamed: 0          0
name_kor_x          0
name_eng            1
producer_kor_x      9
producer_eng_x      1
region_kor          0
region_eng          0
style_kor           0
style_eng           0
apv                 0
ibu               276
price             219
info               21
review_eng        238
mean_average      175
mean_review       221
score             237
mean_weighted     235
dtype: int64

In [ ]:
df.dropna(subset=['review_eng'], inplace=True) # 결측치 제거
df.reset_index(inplace=True)
df.isnull().sum()

index               0
Unnamed: 0          0
name_kor_x          0
name_eng            0
producer_kor_x      9
producer_eng_x      1
region_kor          0
region_eng          0
style_kor           0
style_eng           0
apv                 0
ibu               218
price             173
info               21
review_eng          0
mean_average       29
mean_review        53
score              54
mean_weighted      53
dtype: int64

In [ ]:
s=[]

for i in range(len(df['review_eng'])) :
  rev=df.loc[i,'review_eng']
  rev = rev[1:len(rev)-1]
  rev = rev.replace('\n',' ').replace('] ,', '],') # 줄바꿈 공백으로 변환
  s.append(rev.split('],'))


In [ ]:
len(s[2])

6

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

for i in range(len(s)):
  for j in range(len(s[i])):
    s[i][j] = re.sub('[^a-zA-Z]', ' ', s[i][j]) # 특수문자 제거
    s[i][j] = s[i][j].lower().split() # 대문자 소문자로 변경

    # 불용어 제거
    stops = set(stopwords.words('english'))
    s[i][j] = [word for word in s[i][j] if not word in stops]

    # 어간 추출
    stemmer = nltk.stem.SnowballStemmer('english')
    s[i][j] = [stemmer.stem(word) for word in s[i][j]]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
s[800][2]

['amber',
 'color',
 'excel',
 'foam',
 'nose',
 'bright',
 'candi',
 'lollipop',
 'note',
 'dri',
 'fruit',
 'dri',
 'apricot',
 'tast',
 'sweet',
 'bitter',
 'excel',
 'balanc',
 'malt',
 'note',
 'dri',
 'fruit',
 'sour',
 'herbac',
 'floral',
 'hint',
 'hmm',
 'aftertast',
 'pleasant',
 'long',
 'strong',
 'hop',
 'bitter',
 'hint',
 'pine',
 'full',
 'flavor',
 'beer',
 'reveal',
 'heat',
 'ipa',
 'trend']

In [ ]:
df['review_eng'] = s 

#모델 학습

In [ ]:
yelp=pd.read_csv("/content/drive/MyDrive/review.csv",encoding="utf-8-sig")

In [ ]:
yelp.head()

,abl,absolut,accommod,across,actual,adam,add,addit,afternoon,ago,alcohol,ale,allow,almost,along,alreadi,also,although,alway,amaz,ambianc,amount,anoth,anyon,anyth,anyway,anywher,app,appar,appet,appreci,area,arent,around,arriv,ask,ate,atmospher,attent,austin,...,week,weekend,weird,welcom,well,went,werent,weve,what,whatev,white,whole,wife,will,wine,wing,winter,wish,within,without,wonder,wont,word,work,world,worst,worth,wouldnt,wow,write,wrong,year,yelp,yes,yet,youll,your,yum,yummi,positive
0,0,1,0,0,1,0,1,0,0,2,0,0,0,0,0,0,2,0,0,2,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,...,0,0,0,0,1,2,1,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,True
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,True
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True


In [ ]:
yelp.positive.value_counts()

True     18963
False     4234
Name: positive, dtype: int64

In [ ]:
18963 / len(yelp)

0.8174763978100617

In [ ]:
from sklearn.tree import DecisionTreeClassifier 

train_Y = yelp.positive
train_X = yelp.drop(columns = 'positive')

In [ ]:
model = DecisionTreeClassifier()

In [ ]:
model.fit(train_X, train_Y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
model.predict(train_X)

array([ True,  True,  True, ...,  True,  True,  True])

#감성분석 시작...!

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!pip install -U textblob

In [ ]:
!python -m textblob.download_corpora

In [ ]:
!pip install vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer #vader 용어집 이용
sid = SentimentIntensityAnalyzer()
sentence = 'This is a great VADER Example'
sid.polarity_scores(sentence)

{'compound': 0.6249, 'neg': 0.0, 'neu': 0.549, 'pos': 0.451}

In [ ]:
## 이중리스트 생성

a=[]
for i in range(len(s)):
      a.append(len(s[i]))    # 가로 크기를 저장한 리스트

b = []    # 빈 리스트 생성
 
for i in a:      # 가로 크기를 저장한 리스트로 반복
    line = []    # 안쪽 리스트로 사용할 빈 리스트 생성
    for j in range(i):    # 리스트 a에 저장된 가로 크기만큼 반복
        line.append(0)
    b.append(line) 

In [ ]:
df['neu'] = b; df['pos'] = b; df['neg'] = b; df['compound'] = b # 중립 긍정 부정 복합 열 이중 리스트로 만들어줌

for i in range(len(s)):
  for j in range(len(s[i])):
     df['neu'][i][j] = [sid.polarity_scores(v)['neu'] for v in df['review_eng'][i][j]]
     df['pos'][i][j] = [sid.polarity_scores(v)['pos'] for v in df['review_eng'][i][j]]
     df['neg'][i][j] = [sid.polarity_scores(v)['neg'] for v in df['review_eng'][i][j]]
     df['compound'][i][j] = [sid.polarity_scores(v)['compound'] for v in df['review_eng'][i][j]]
    

In [ ]:
df.to_csv('example.csv',encoding='utf-8-sig')